In [ ]:
# 소프트맥스 회귀를 로우-레벨과 F.cross_entropy 를 사용하여 구현

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)


In [2]:
# 훈련 데이터와 레이블을 텐서로 선언
x_train = [ # 4가지 특성, 8가지 샘플이 존재
    [1, 2, 1, 1],
    [2, 1, 3, 2],
    [3, 1, 3, 4],
    [4, 1, 5, 5],
    [1, 7, 5, 5],
    [1, 2, 5, 6],
    [1, 6, 6, 6],
    [1, 7, 7, 7]
]
y_train = [2, 2, 2, 1, 1, 1, 0, 0] # 각 샘플에 대한 레이블, 0, 1, 2 값이 존재 -> 3개의 클래스
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [ ]:
# 소프트맥스 회귀 구현하기(로우-레벨)

In [3]:
# x_train, y_train 크기 확인
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [5]:
# 최종 사용할 레이블은 y_train 에서 원-핫 인코딩을 한 결과이여야 한다
# 클래수 3개 -> y_train 에 원-핫 인코딩을 한 결과는 8 X 3 크기

y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
print(y_one_hot.shape)

torch.Size([8, 3])


In [6]:
# W 과 b 선언
# W 행렬 크기 : x_trian (8 X 4), y_one_hot (8 X 3) 이므로 (4 X 3) 이다

# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

In [7]:
# F.softmax 와 torch.log()를 사용하여 가설과 비용 함수를 정의하고, 1000번 에포크 수행

nb_epochs = 1000
for epoch in range(nb_epochs+1):

    # H(x) 정의
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)

    # Cost
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost 로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번 마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [ ]:
# 소프트맥스 회귀 구현하기(하이-레벨)

# F.cross_entropy() 사용하여 비용 함수 구현
# F.cross_entropy() 는 소프트맥스 함수 포함하기 때문에 가설에서 소프트맥스 함수를 사용할 필요가 없음

# 위와 동일한 x_train 과 y_train 사용

In [8]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

# 1000번 에포크 수행
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 설정 필요 X
    # Cost
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost 로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568255
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [9]:
# 소프트맥스 회귀 nn.Module 로 구현하기

# 선형 회귀에서 구현에 사용했던 nn.Linear() 사용
# output_dim = 클래스의 개수

In [10]:
# 모델을 선언 및 초기화, 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3
model = nn.Linear(4, 3)

In [11]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost
    cost = F.cross_entropy(prediction, y_train)

    # cost 로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.616785
Epoch  100/1000 Cost: 0.658891
Epoch  200/1000 Cost: 0.573443
Epoch  300/1000 Cost: 0.518151
Epoch  400/1000 Cost: 0.473265
Epoch  500/1000 Cost: 0.433516
Epoch  600/1000 Cost: 0.396563
Epoch  700/1000 Cost: 0.360914
Epoch  800/1000 Cost: 0.325392
Epoch  900/1000 Cost: 0.289179
Epoch 1000/1000 Cost: 0.254148


In [12]:
# 소프트맥스 회귀 클래스로 구현하기

# nn.Module 을 상속받은 클래스로 구현
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)

    def forward(self, x):
        return self.linear(x)

In [13]:
model = SoftmaxClassifierModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost
    cost = F.cross_entropy(prediction, y_train)

    # cost 로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 2.637636
Epoch  100/1000 Cost: 0.647903
Epoch  200/1000 Cost: 0.564643
Epoch  300/1000 Cost: 0.511043
Epoch  400/1000 Cost: 0.467249
Epoch  500/1000 Cost: 0.428281
Epoch  600/1000 Cost: 0.391924
Epoch  700/1000 Cost: 0.356742
Epoch  800/1000 Cost: 0.321577
Epoch  900/1000 Cost: 0.285617
Epoch 1000/1000 Cost: 0.250818
